In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType,BooleanType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/17 15:57:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/ShopInfo')

In [4]:
sample_data = data.select('value').take(5)

In [5]:
data_sample = [loads(item.value) for item in sample_data]

In [6]:
data_sample[3]

{'id': 251877,
 'store_id': 238909,
 'name': 'Giày dép RYNA',
 'icon': 'https://vcdn.tikicdn.com/ts/seller/95/57/12/424635ac79ea7cb66baf8b2ccca69e1c.jpg',
 'url': 'https://tiki.vn/cua-hang/giay-dep-ryna',
 'is_official': False,
 'store_level': 'NONE',
 'is_followed': False,
 'info': [{'type': 'review', 'title': '4.3 / 5', 'sub_title': '44 đánh giá'},
  {'type': 'normal', 'title': '13', 'sub_title': 'Người theo dõi'}],
 'avg_rating_point': 4.3182,
 'review_count': 44,
 'total_follower': 13,
 'days_since_joined': 339}

In [7]:
schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("store_id",IntegerType(),True), 
    StructField("name",StringType(),True), 
    StructField("icon",StringType(),True),
    StructField("url",StringType(),True), 
    StructField("is_official",BooleanType(),True),
    StructField("store_level",StringType(),True),
    StructField('is_followed', BooleanType(),True),
    StructField('avg_rating_point', FloatType(),True),
    StructField('review_count', IntegerType(),True),
    StructField("total_follower",IntegerType(),True), 
    StructField("days_since_joined",IntegerType(),True), 
])

In [8]:
df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

In [11]:
df.createOrReplaceTempView('shopinfo')

In [12]:
df.write.mode('append').parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')